# Test Code for the Galactic Plane Figure of Merit

Code adapted from an example by Lynne Jones

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import rubin_sim.maf as maf

import galPlaneFootprintMetric, galPlaneTimePerFilterMetric, transientTimeSamplingMetric

Load the baseline v2.0 as a test case OpSim:

In [2]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)
opsim_db = maf.OpsimDatabase(opsim_fname)

/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


Taking an example sky location within the Galactic Bulge:

In [3]:
test_ra = (17.0 + 57.0/60.0 + 34.0/3600.0)*15.0
test_dec = (29.0 + 13.0/60.0 + 15.0/3600.0)*-1.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

In [8]:
class galPlaneFoM(maf.BaseMetric):
    """Figure of Merit to evaluate the combination of survey footprint, cadence and filter selection for suitability 
    for Galactic Plane science"""
    
    def __init__(self, cols=['fieldRA','fieldDec','filter',
                             'observationStartMJD','visitExposureTime','fiveSigmaDepth'],
                       metricName='galPlaneFoM',
                       **kwargs):
        
        self.ra_col = 'fieldRA'
        self.dec_col = 'fieldDec'
        self.m5Col = 'fiveSigmaDepth'
        self.filters = ['u','g', 'r', 'i', 'z', 'y']
        self.magCuts = {'u': 22.7, 'g': 24.1, 'r': 23.7, 'i': 23.1, 'z': 22.2, 'y': 21.4}
        self.mjdCol = 'observationStartMJD'
        self.exptCol = 'visitExposureTime'
        
        super().__init__(col=cols, metricName=metricName)


    def run(self, dataSlice, slicePoint=None):

        # Survey footprint metric produces a single value
        footprint_metric = galPlaneFootprintMetric.galPlaneFootprintMetric()
        footprint_metric_data = footprint_metric.run(dataSlice, slicePoint)
        
        # Cadence metric produces a set of four values for four time categories
        cadence_metric = transientTimeSamplingMetric.transientTimeSamplingMetric()
        cadence_metric_data = cadence_metric.run(dataSlice, slicePoint)
        
        # Filter selection metric produces a single value
        filters_metric = galPlaneTimePerFilterMetric.galPlaneTimePerFilter()
        filters_metric_data = filters_metric.run(dataSlice, slicePoint)
        
        # Note: Consider summing cadence_metric_values over all variability categories
        # to produce a single FoM value?
        fom = 0.0
        for t_metric in cadence_metric_data.metric_values:
            fom += footprint_metric_data.data * t_metric * filters_metric_data.data

        return fom


In [9]:
mymetric = galPlaneFoM()
sqlconstraint = None
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)
g.runAll()

Querying database observations with no constraint for columns ['rotSkyPos', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'fieldDec', 'visitExposureTime', 'filter'].
Found 2086980 visits
Running:  ['test_metric']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [10]:
bundle.metricValues

masked_array(data=[0.0635873456554306],
             mask=[False],
       fill_value=-666.0)